# TODO

- [ ] Figure out how to do hyperparam search
- [ ] Move code into separate `.py` files
- [ ] Run experiments with 4 optimizers (SGD, HeavyBall, Adam, Adam w/o momentum) and 2 datasets (MNIST, FashionMNIST)

In [22]:
from __future__ import annotations

import torch as t
from torch.optim import lr_scheduler, SGD, Adam


from src.heavy_ball import HeavyBall
from src.simple_cnn import SimpleCNN, DatasetSimpleCNN
from src.training import train

# from kfac.preconditioner import KFACPreconditioner

## Model, data, and training setup

In [2]:
model = SimpleCNN()
ds_m = DatasetSimpleCNN.load() # MNIST
ds_f = DatasetSimpleCNN.load(fashion_mnist=True) # FashionMNIST
BATCH_SIZES: list[int] = (2 ** t.arange(4, 14)).tolist()

## Experiments

### Optimizer: SGD

In [4]:
model = SimpleCNN()
#TODO tune LR (and other hyperparams for other optimizers)
lr = 1e-3
optimizer = t.optim.SGD(model.parameters(), lr)
#TODO fine-tune scheduler
scheduler = t.optim.lr_scheduler.LinearLR(optimizer)
# trs: dict[int, TrainingResult] = {
#     batch_size: train(model, ds_m,)
# }
# tr = train(model, ds_m, optimizer, BATCH_SIZES[0])


### Optimizer: HeavyBall

In [20]:
model = SimpleCNN()
lr = 1e-3
# optimizer = t.optim.SGD(model.parameters(), lr, momentum=0.8)
optimizer = HeavyBall(model.parameters(), lr=lr, momentum=0.8)
tr = train(model, ds_m, optimizer, BATCH_SIZES[0])

Step 0: running_loss=6.9172, test_acc=0.00%
Step 100: running_loss=3.6794, test_acc=88.17%
Step 200: running_loss=0.4058, test_acc=91.10%
Step 300: running_loss=0.3104, test_acc=91.73%
Step 400: running_loss=0.2511, test_acc=92.37%
Step 500: running_loss=0.3058, test_acc=93.34%
Step 600: running_loss=0.3057, test_acc=93.75%
Step 700: running_loss=0.1894, test_acc=94.44%
Step 800: running_loss=0.2044, test_acc=93.90%
Step 900: running_loss=0.2049, test_acc=95.92%
Step 1000: running_loss=0.1795, test_acc=95.92%
Step 1100: running_loss=0.1538, test_acc=96.27%
Step 1200: running_loss=0.1458, test_acc=95.93%
Step 1300: running_loss=0.1314, test_acc=95.66%
Step 1400: running_loss=0.1081, test_acc=96.32%
Step 1500: running_loss=0.1201, test_acc=96.75%
Step 1600: running_loss=0.1436, test_acc=95.60%
Step 1700: running_loss=0.1551, test_acc=96.79%
Step 1800: running_loss=0.1134, test_acc=96.71%


KeyboardInterrupt: 

### Optimizer: Adam with momentum
    

In [ ]:
model = SimpleCNN()
lr = 1e-3
optimizer = t.optim.Adam(model.parameters(), lr)
tr = train(model, ds_m, optimizer, BATCH_SIZES[0])

### Optimizer: Adam without momentum

In [29]:
model = SimpleCNN()
lr = 1e-3
optimizer = t.optim.Adam(model.parameters(), lr, betas=(0, 0))
# tr = train(model, ds_m, optimizer, BATCH_SIZES[0])
optimizer.param_groups[0]["betas"]

(0, 0)